<a href="https://colab.research.google.com/github/geetesh-gupta/google_colab_codes/blob/master/UploadFromDriveToFirebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Authenticate Google Drive account
from google.colab import drive
drive.mount("/content/drive")

In [0]:
# Authenticate Google Colaboratory
from google.colab import auth
auth.authenticate_user()

In [0]:
# Firebase Project ID
project_id = 'wordpress-330de'
!gcloud config set project {project_id}

In [0]:
# List all the buckets in the project
!gsutil ls

In [0]:
# Set the bucket to work with from the previous list
# Remember to remove the gs:// part
bucket_name = 'wordpress-330de.appspot.com'

In [0]:
# Set the drive folder path
drive_folder = '/content/drive/My Drive/Celeb Offer Videos/'

In [0]:
# Set the cloud folder path
cloud_folder = '/celeb'
# cloud_folder = '/celeb_images'

# content type extension
extension = '.mp4'
# extension = '.jpg'

In [0]:
import os, re

# Loop through the drive folders recursively
for (rootDir, subDirs, files) in os.walk(drive_folder):
  # Get the folder name
  splitRootDirPath = rootDir.split('/')
  dirfilename = splitRootDirPath[-1]

  # A Fix for those who have extra subfolder named Offer Video. Uncomment only if needed
  # if splitRootDirPath[-1] == "Offer Video":
  #   dirfilename = splitRootDirPath[-2]
    
  # Get all the video paths 
  filepaths = []
  for file in files:
      if re.match('.*'+extension, file):
          filepaths.append(file)
  # print(filepaths)
  
  # Create filenames based on the folder name
  filenames = []
  if len(filepaths) > 1:
      for i, file in enumerate(filepaths):
          filenames.append(dirfilename.replace(' ', '_') + '_' + str(i+1))
  elif len(filepaths) == 1:
      filenames.append(dirfilename.replace(' ', '_'))
  # print(filenames)

  # Copy the files
  for i, file in enumerate(filepaths):
      filepath = '"'+rootDir + '/' + file+'"'
      filename = '"'+filenames[i]+extension+'"'

      already_present = !gsutil -q stat gs://{bucket_name}{cloud_folder}/{filename}
      # Replace the "/celeb" part with whatever folder or path you want to use
      if already_present == 1:
        !gsutil -m cp -r {filepath} gs://{bucket_name}{cloud_folder}/{filename}
      else:
        print(filename + " Already Present")

In [0]:
# To transfer std out to local string variable
from io import StringIO
import sys

old_stdout = sys.stdout
result = StringIO()
sys.stdout = result

# Output the firebase folder filenames
!gsutil ls gs://{bucket_name}{cloud_folder}/*{extension}

sys.stdout = old_stdout

result_string = result.getvalue()


filenames = [path.split('/')[-1] for path in result_string.split('\n')]
print(filenames)

# Base url to access the content
base_url_prefix = "https://firebasestorage.googleapis.com/v0/b/"+bucket_name+"/o"+cloud_folder+"%2F"

# May change with type of data
base_url_suffix = "?alt=media"

fileurls = [base_url_prefix + filename + base_url_suffix for filename in filenames]
print(fileurls)


In [0]:
# Create spreadsheet
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default()) 

sheet_name = 'Celeb_Videos_URLs'

# Uncomment if sheet not present in the drive
# gc.create(sheet_name)

# Open the sheet
worksheet = gc.open(sheet_name).get_worksheet(0)

# First Row Headings
# worksheet.append_row(["Name", "URL"])

old_records = worksheet.get_all_values()
old_records.pop(0)

# Add row wise data
for i, (filename, fileurl) in enumerate(zip(filenames, fileurls)):
  if filename == "":
    continue
  file = [filename, fileurl]
  if file not in old_records:
    print(file)
    worksheet.append_row([filename, fileurl])

In [0]:
!pip install gspread_formatting
import gspread_formatting

In [0]:
# Sort Sheet 
from gspread_formatting import *
rows = worksheet.get_all_values()
rows.pop(0)
rows_format = []
for i, row in enumerate(rows):
  try:
    row_format = []
    for j in range(len(row)):
      cell_label = chr(ord('A')+j)+str(i+2)
      row_format.append(get_effective_format(worksheet, cell_label))
    rows_format.append(row_format)
  except Exception:
    continue
    
sorted_pairs = sorted(zip(rows, rows_format), key=lambda pair: pair[0][0])
tuples = zip(*sorted_pairs)
rows, rows_format = [list(tuple) for tuple in  tuples]

# # gc.open(sheet_name).add_worksheet()
# # # worksheet = gc.open(sheet_name).get_worksheet(2)
for i, row in enumerate(rows):
  for j in range(len(row)):
    cell_label = chr(ord('A')+j)+str(i+2)
    format_cell_range(worksheet, cell_label, rows_format[i][j])
    worksheet.update_acell(cell_label, row[j])
